In [1]:
import torch
import torch.nn.functional as F

torch.manual_seed(27)

# Example tensor of shape (1, 1, H, W)
DIM = 4
W = torch.rand(DIM, DIM)
P1 = torch.rand(int(DIM/2), DIM)
P2 = torch.rand(DIM, int(DIM/2))



In [2]:
P1@W#@P2

tensor([[1.5872, 1.1626, 1.6426, 0.9742],
        [0.4341, 0.6854, 0.8121, 0.5155]])

In [3]:
DIM1 = 12
DIM2 = 4
W = torch.rand(DIM1, DIM2)
P1 = torch.rand(int(DIM1/2), DIM1)
P2 = torch.rand(DIM2, int(DIM2/2))

In [4]:
r = P1@W@P2
print(W.shape)
print(r.shape)
r

torch.Size([12, 4])
torch.Size([6, 2])


tensor([[5.6443, 5.0776],
        [6.5575, 5.7987],
        [7.7880, 7.0714],
        [7.1346, 6.4522],
        [6.2099, 5.7591],
        [7.6796, 6.9450]])

In [13]:
xb = 512
yb = 2
xs = 256
ys = 4
W = torch.rand(xb, yb)
P1 = torch.rand(xs, xb)
P2 = torch.rand(yb, ys)

In [14]:
r = P1@W@P2
print(W.shape)
print(r.shape)
r

torch.Size([32768, 512])
torch.Size([8, 4])


tensor([[2120721.7500, 2136134.0000, 2145541.0000, 2163374.5000],
        [2120979.0000, 2136126.7500, 2145764.5000, 2163616.0000],
        [2119225.2500, 2134346.5000, 2143875.0000, 2161988.0000],
        [2120395.5000, 2135539.7500, 2145210.5000, 2162987.0000],
        [2114497.0000, 2129851.2500, 2139335.2500, 2157119.0000],
        [2112680.0000, 2127787.2500, 2137059.7500, 2155093.0000],
        [2109990.0000, 2125115.7500, 2134586.0000, 2152721.0000],
        [2109821.0000, 2125131.0000, 2134528.7500, 2152302.2500]])

In [3]:
from lizrd.core.misc import Linear


Linear(
    4, #xs
    8, #xb
    init_type = "kaiming_uniform",
    init_scale = 1.0
)

Linear(in_features=4, out_features=8, bias=False)